# IA Challenge TSE Team 9AM

In [ ]:
# Goal: Predict the accumulated daily rainfall 

Set-up Cloud Environment

In [ ]:
from google.colab import files

from google.colab import drive
drive.mount('/content/gdrive')

uploaded = files.upload() # don't forget to turn down your shields to authorize the upload

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


KeyboardInterrupt: ignored

Load Data

In [ ]:
!kaggle competitions download -c defi-ia-2022

 54% 5.00M/9.28M [00:01<00:01, 3.72MB/s]
100% 9.28M/9.28M [00:01<00:00, 6.63MB/s]
  0% 0.00/2.14M [00:00<?, ?B/s]
100% 2.14M/2.14M [00:00<00:00, 61.4MB/s]
  0% 0.00/1.23M [00:00<?, ?B/s]
100% 1.23M/1.23M [00:00<00:00, 128MB/s]
100% 25.5M/25.5M [00:01<00:00, 10.5MB/s]

 61% 5.00M/8.22M [00:01<00:00, 4.05MB/s]
100% 8.22M/8.22M [00:01<00:00, 6.53MB/s]
  0% 0.00/1.74M [00:00<?, ?B/s]
100% 1.74M/1.74M [00:00<00:00, 57.8MB/s]
  0% 0.00/918k [00:00<?, ?B/s]
100% 918k/918k [00:00<00:00, 29.5MB/s]
  0% 0.00/1.05M [00:00<?, ?B/s]
100% 1.05M/1.05M [00:00<00:00, 34.5MB/s]
100% 65.3M/65.3M [00:02<00:00, 29.2MB/s]

  0% 0.00/10.8k [00:00<?, ?B/s]
100% 10.8k/10.8k [00:00<00:00, 9.47MB/s]
  0% 0.00/306k [00:00<?, ?B/s]
100% 306k/306k [00:00<00:00, 98.3MB/s]
  0% 0.00/343k [00:00<?, ?B/s]
100% 343k/343k [00:00<00:00, 113MB/s]
  0% 0.00/1.43M [00:00<?, ?B/s]
100% 1.43M/1.43M [00:00<00:00, 46.9MB/s]
 90% 18.0M/20.0M [00:01<00:00, 5.83MB/s]
100% 20.0M/20.0M [00:01<00:00, 15.0MB/s]
  0% 0.00/2.12k [00:00<?

Load packages

In [ ]:
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline
%matplotlib inline
plt.style.use('dark_background')
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
import tensorflow as tf

Set-up train test sets

In [ ]:
# Lecture of the data frame
X_train = pd.read_csv('X_station_train.csv', delimiter=',')
Y_train = pd.read_csv('Y_train.csv', delimiter=',')

X_test = pd.read_csv('X_station_test.csv', delimiter=',')
Y_test = pd.read_csv('Id_month_test.csv', delimiter=',')

# Display the dataset

print('train set: {0}'.format(X_train.shape))
X_train.head(10)

print('test set: {0}'.format(X_test.shape))
X_test.head(10)

FileNotFoundError: ignored

In [ ]:
Y_train

Extract day and month from train set

In [ ]:
X_train['month']=pd.DatetimeIndex(X_train['date']).month
X_train['day']=pd.DatetimeIndex(X_train['date']).day

In [ ]:
# impute missing values by replacing them with values on their previous day.

def fill_missing(df):
    one_day = 1
    for row in range(df.shape[0] - 1):
        for col in range(df.shape[1]):
            if (pd.isna(df.iloc[row,col])):
                df[row,col] = df[row-one_day,col]
fill_missing(X_test)
fill_missing(X_train)

X_test.isnull().sum()
X_train.isnull().sum()

In [ ]:
X_train

In [ ]:
X_test

Extract station id, day and month from test set

In [ ]:
x= X_test['Id'].str.split('_', expand = True)
X_test["station"]=x[0]
X_test["day"]=x[1]
X_test["hour_id"]=x[2]
X_test.drop(['hour_id'], axis=1)

In [ ]:
X_test

Group by day, month and station id for train set : Ground truth of monthly means for precipation

In [ ]:
X_train = X_train.rename(columns={"number_sta": "station"})

In [ ]:
X_test_tmp = X_test.groupby(by = ['month', 'station'] ).size()
X_test_tmp = X_test.groupby(by = ['day', 'station'] ).mean()
X_test_tmp = X_test.groupby(by = ['month', 'station'] ).mean()
pd.DataFrame(X_test_tmp)

Group by month and station id for test set 

In [ ]:
X_train_tmp = X_train.groupby(by = ['month', 'station'] ).size()
X_train_tmp = X_train.groupby(by = ['day', 'station']).mean()
X_train_tmp = X_train.groupby(by = ['month', 'station']).mean()
X_train_tmp = X_train_tmp.iloc[:, :-1]

In [ ]:
X_train_tmp

In [ ]:
#X_test_tmp = X_test_tmp.fillna(X_test_tmp.mean())
#X_train_tmp = X_train_tmp.fillna(X_train_tmp.mean())

In [ ]:
X_train.isnull().sum()
Y_train.isnull().sum()

X_test.isnull().sum()
Y_test.isnull().sum()

In [ ]:
Y_train

In [ ]:
Y_train['day']=pd.DatetimeIndex(Y_train['date']).day
Y_train['month']=pd.DatetimeIndex(Y_train['date']).month
Y_train = Y_train.rename(columns={"number_sta": "station"})

Y_train = Y_train.groupby(by = ['month', 'station'] ).size()
Y_train = Y_train.groupby(by = ['day', 'station']).mean()
Y_train = Y_train.groupby(by = ['month', 'station']).mean()

Y_train = pd.DataFrame(Y_train)

In [ ]:
Y_train = Y_train.rename(columns={Y_train.columns[0]: 'precip'})

In [ ]:
Y_train

In [ ]:
# Pre-process data: Scaling for convergence

sc = MinMaxScaler(feature_range = (0, 1))
X_train_scaled = sc.fit_transform(X_train_tmp)
X_test_scaled = sc.fit_transform(X_test_tmp )

# Reshaping for computation

X_train_reshaped= X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_train_reshaped.shape

X_test_reshaped= X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])
X_test_reshaped.shape


In [ ]:
X_test_reshaped

In [ ]:
X_train_reshaped

In [ ]:
model_lstm = tf.keras.layers.LSTM(
    4,
    activation="tanh",
    recurrent_activation="sigmoid",
    use_bias=True,
    kernel_initializer="glorot_uniform",
    recurrent_initializer="orthogonal",
    bias_initializer="zeros",
    unit_forget_bias=True,
    kernel_regularizer=None,
    recurrent_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    recurrent_constraint=None,
    bias_constraint=None,
    dropout=0.25,
    recurrent_dropout=0.25,
    return_sequences=False,
    return_state=False,
    go_backwards=False,
    stateful=False,
    time_major=False,
    unroll=False,
)


In [ ]:
# Model Architecture 
## Model 1: E1D1 ==> Sequence to Sequence Model with one encoder layer and one decoder layer

# E1D1
# n_features ==> no of features at each timestep in the data.
#

inputs = keras.layers.Input(shape=X_train_reshaped, Y_train))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(1)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(inputs,decoder_outputs1)

#

model_e1d1.summary()

In [ ]:
# Model Architecture 
## Model 1: E1D2 ==> Sequence to Sequence Model with one encoder layer and one decoder layer

# E1D2
# E2D2
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(1,8))
encoder_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)
#
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_e2d2.summary()

In [ ]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[reduce_lr])
model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2=model_e2d2.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[reduce_lr])

In [ ]:
pred_e1d1=model_e1d1.predict(X_test)
pred_e2d2=model_e2d2.predict(X_test)

In [ ]:
for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]
    pred1_e1d1[:,:,index]=scaler.inverse_transform(pred1_e1d1[:,:,index])
    pred_e1d1[:,:,index]=scaler.inverse_transform(pred_e1d1[:,:,index])
    pred1_e2d2[:,:,index]=scaler.inverse_transform(pred1_e2d2[:,:,index])
    pred_e2d2[:,:,index]=scaler.inverse_transform(pred_e2d2[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

In [ ]:
from sklearn.metrics import mean_absolute_error
for index,i in enumerate(train_df.columns):
  print(i)
  for j in range(1,6):
    print("Day ",j,":")
    print("MAE-E1D1 : ",mean_absolute_error(y_test[:,j-1,index],pred1_e1d1[:,j-1,index]),end=", ")
    print("MAE-E2D2 : ",mean_absolute_error(y_test[:,j-1,index],pred1_e2d2[:,j-1,index]))
  print()
  print()

In [ ]:
https://colab.research.google.com/drive/1JwDl3HZ9SfvV5crtjbuFPaU_T7Q1y34w?usp=sharing

In [ ]:
inputs = keras.layers.Input(shape=X_train_reshaped, Y_train))
lstm_out = keras.layers.LSTM(32)(inputs)
outputs = keras.layers.Dense(1)(lstm_out)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model.summary()


output = model_lstm(X_train_reshaped)

In [ ]:
output

In [ ]:
plt.clf()
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'y', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, 'g', label='Training acc')
plt.plot(epochs, val_acc, 'y', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()